# CulturoGemma A Gemma agent
This notebook details the journey of creating an AI chat agent powered by the Gemma model, designed to engage in conversations across multiple languages and with cultural sensitivity. We'll walk through the code, explaining each component and the reasoning behind the design choices.

## Introduction
The goal of this project is to build an AI assistant capable of understanding and responding to users in various languages, while also being mindful of cultural nuances. This involves leveraging a powerful language model like Gemma and integrating external tools for translation and information retrieval.
This will enable us to give the models like Gemma for `Unlocking Global Communication`

## Setting Up the Environment
Before we begin, let's ensure our environment is set up correctly. You'll need to install the following Python libraries:
```bash
!pip install transformers torch tavily-py google-cloud-translate wikipedia streamlit
```

* **`transformers`**: Provides access to pre-trained language models like Gemma.
* **`torch`**:  A fundamental library for tensor computation, essential for deep learning models.
* **`tavily-py`**: A library for interacting with the Tavily API, which allows us to perform web searches and extract content from URLs.
* **`google-cloud-translate`**:  The Google Cloud Translation API client library for translating text between languages.
* **`wikipedia`**: A library to easily access and search Wikipedia content.
* **`streamlit`**: A framework for building interactive web applications with Python.

You'll also need API keys for Tavily(its free) and Google Cloud Translation. It's recommended to store these securely as environment variables rather than hardcoding them in your script(kaggle, colab secrets).

In [ ]:
!pip install -q wikipedia tavily-python

In [ ]:
import os
import io
import re
import json
import torch
import wikipedia
from tavily import TavilyClient
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Set the environment variables for Kaggle and tavily search.
# from kaggle_secrets import UserSecretsClient if you use kaggle
# from google.colab import userdata if you use google colab
#import getpass if you use jupyter notebook
os.environ["KAGGLE_USERNAME"] = "your-username"# or UserSecretsClient().get_secret(KAGGLE_USERNAME) or userdata.get(KAGGLE_USERNAME) or getpass.getpass("Enter your KAGGLE_USERNAME: ")
os.environ["KAGGLE_KEY"] = "kaggle-api-key" # or UserSecretsClient().get_secret(KAGGLE_KEY) or userdata.get(KAGGLE_KEY) or getpass.getpass("Enter your  KAGGLE_KEY: ")
os.environ["TAVILY_KEY"] = "tavily-api-key" # or UserSecretsClient().get_secret(TAVILY_KEY) or userdata.get(TAVILY_KEY) or getpass.getpass("Enter your TAVILY_KEY: ")

In [ ]:
# Initialize API clients using environment variables
try:
    tavily_client = TavilyClient(api_key=os.environ["TAVILY_KEY"])
except KeyError:
    print("Error: TAVILY_KEY environment variable not set.")
    tavily_client = None

**Explanation:**

* We import necessary libraries.
* We initialize the Tavily and Google Cloud Translate clients using API keys retrieved from environment variables. **Remember to replace `'path/to/your/service_account_key.json'` with the actual path to your Google Cloud service account key file.**

## Defining Function Calls

Our agent needs to interact with the outside world to provide comprehensive answers. We achieve this through function calls, which allow the agent to trigger specific actions.

**Functions and why we use them**

Ok, So our functions can be anything, I mean anything but as we want to use this model for **expanding global impact of gemma** we go with these:
-  1- Web search and URL fetching with tavily search api: This enables the model to search the entire web and gather information about the topic in the conversation
-  2- wikipedia: This act like our search functions but only limited to wikipedia as its source of information, Still very useful
-  3- Google translate functions with google cloud api(Not tested): These functions can help if the used language is so far from models knowledge.

```python
def get_answer_from_tavily(query):
    """Searches the web using Tavily and returns the answer."""
    if tavily_client is None:
        return {"error": "Tavily client not initialized."}
    try:
        answer = tavily_client.qna_search(query=query)
        return {"web_result": answer}
    except Exception as e:
        logging.error(f"Error fetching answer from Tavily: {e}")
        return {"error": f"Error fetching answer: {str(e)}"}

def fetch_url(urls):
    """Fetches content from a given URL using Tavily."""
    if tavily_client is None:
        return {"error": "Tavily client not initialized."}
    try:
        extract_response = tavily_client.extract(urls=urls)
        for result in extract_response.get("results", []):
            return {"web_url": result["url"], "extracted_content": result["raw_content"]}
        return {"error": "No results found for the given URL"}
    except Exception as e:
        logging.error(f"Error fetching URL content: {e}")
        return {"error": f"Error fetching URL: {str(e)}"}

def search_wikipedia(query):
    """Searches Wikipedia for a given query and returns a summary."""
    try:
        return {"wikipedia_summary": wikipedia.summary(query, sentences=3)}
    except wikipedia.exceptions.PageError:
        return {"error": f"Wikipedia page not found for query: {query}"}
    except wikipedia.exceptions.DisambiguationError as e:
        return {"error": f"Multiple Wikipedia pages found for query: {query}. Be more specific.", "possible_titles": e.options}
    except Exception as e:
        logging.error(f"Error searching Wikipedia: {e}")
        return {"error": f"Error searching Wikipedia: {str(e)}"}

def translate_to_english(prompt):
  """Translates the given prompt to English, handling potential errors."""
  try:
    if use_google_translate:
      result = translate_client.translate(prompt, target_language='en')
      return result['translatedText']
    else:
      return None  # Return None if translation is disabled
  except GoogleAPIError as e:
    print(f"Error translating to English: {e}")
    return prompt  # Return the original prompt on failure

def translate_to_target(prompt):
  """Translates the given prompt to the previously detected target language, handling errors."""
  global detected_language
  try:
    if use_google_translate and detected_language:
      result = translate_client.translate(prompt, target_language=detected_language)
      return result['translatedText']
    else:
      return "Target language not detected Or google cloud api not provided. Please call detect_language() first and `use_google_translate=True`."
  except GoogleAPIError as e:
    print(f"Error translating to target language: {e}")
    return prompt  # Return the original prompt on failure
```


In [ ]:
def get_answer_from_tavily(query):
    """ This function gets query as str and passes the extracted answer as str"""
    try:
        answer = tavily_client.qna_search(query=query)
        # Assuming the response has an 'answer' key
        return answer
    except Exception as e:
        print(f"Error fetching answer from Tavily: {e}")
        return f"Error fetching answer: {str(e)}"

def fetch_url(urls):
    """ this function fetches the user url and returns the extracted content"""
    try:
        extract_response = tavily_client.extract(urls=urls)
        for result in extract_response.get("results", []):
            return result["url"], result["raw_content"]
        return "No results found", "No content found"
    except Exception as e:
        print(f"Error fetching URL content: {e}")
        return f"Error fetching URL: {str(e)}", f"Error fetching content: {str(e)}"

In [ ]:
use_google_translate = False
if use_google_translate:
    from google.cloud import translate_v2 as translate
    from google.api_core.exceptions import GoogleAPIError
    # Create a client with your credentials (replace with your service account key file)
    translate_client = translate.Client.from_service_account_json('path/to/your/service_account_key.json')

detected_language = None  # Initialize a global variable to store the detected language

def detect_language(prompt):
    """Detects the language of the given prompt, handling potential errors."""
    global detected_language
    try:
        # Check if translation functionality is enabled
        if use_google_translate:
            result = translate_client.detect_language(prompt)
            detected_language = result['language']
            return detected_language
        else:
            # Return None or a default language if translation is disabled
            return None
    except GoogleAPIError as e:
        print(f"Error detecting language: {e}")
        return None

def translate_to_english(prompt):
  """Translates the given prompt to English, handling potential errors."""
  try:
    if use_google_translate:
      result = translate_client.translate(prompt, target_language='en')
      return result['translatedText']
    else:
      return None  # Return None if translation is disabled
  except GoogleAPIError as e:
    print(f"Error translating to English: {e}")
    return prompt  # Return the original prompt on failure

def translate_to_target(prompt):
  """Translates the given prompt to the previously detected target language, handling errors."""
  global detected_language
  try:
    if use_google_translate and detected_language:
      result = translate_client.translate(prompt, target_language=detected_language)
      return result['translatedText']
    else:
      return "Target language not detected Or google cloud api not provided. Please call detect_language() first and `use_google_translate=True`."
  except GoogleAPIError as e:
    print(f"Error translating to target language: {e}")
    return prompt  # Return the original prompt on failure

**Explanation:**

* We define functions for each specific task:
    * `get_answer_from_tavily`: Searches the web using the Tavily API.
    * `fetch_url`: Extracts content from a given URL using the Tavily API.
    * `search_wikipedia`: Searches for information on Wikipedia.
    * `translate_to_english`: Translates text to English using Google Cloud Translate.
    * `translate_to_target`: Translates text to a specified target language using Google Cloud Translate.
* Each function includes error handling to gracefully manage potential issues (e.g., API errors, network problems).
* We check if the API clients are initialized before attempting to use them.

## Crafting the System Prompt

The system prompt guides the behavior of the language model. It tells the model its role, how to interact with users, and how to use the function calls.

```python
"""Your task is to assist users by searching for information and providing accurate responses.

For questions requiring current or factual information, you must first search using this format:
FUNCTION_CALL: get_web_result("search query")

EXAMPLES:
User: What is Thanksgiving?
Assistant: FUNCTION_CALL: get_web_result("What is Thanksgiving holiday history traditions")

User: What's happening in Iran?
Assistant: FUNCTION_CALL: get_web_result("current news Iran")

Other functions that you have access and you can use if needed:
For searching Wikipedia: FUNCTION_CALL: search_wikipedia("topic to search on wikipedia")
EXAMPLE:
user: What is Chaharshanbe Suri
Assistant: FUNCTION_CALL: search_wikipedia("Chaharshanbe Suri")

For fetching content from a specific URL(from user if provided): FUNCTION_CALL: fetch_url("URL")
For translating user input into English (for your better understanding): FUNCTION_CALL: get_translation("user's input in their language")
For translating your response into the user's language: FUNCTION_CALL: translate("your response in English")

Current conversation:
"""
```
Keep in mind we make it in this format so we don't get to see the system prompt at the beggining of each response.
And we do all these things because gemma dose not accept system role.

**Explanation:**

* The system prompt clearly defines the AI's role as a helpful, multilingual, and culturally aware assistant.
* It explicitly instructs the AI on how to use the defined function calls.
* It emphasizes cultural sensitivity, respect for different languages.

Note: You can play with the system prompt to see what works better, This version works well after some trail and errors I made this.

## Building the `CulturoGemma` Class

The `CulturoGemma` class encapsulates the logic for interacting with the language model, managing conversations, and executing function calls.

**The model:**
* I used the multilingual DPO fine-tuned version of Gemma2 model family, More information can be found on the fine-tuning [notebook](https://www.kaggle.com/code/mahdiseddigh/multilingual-dpo-fine-tuning-gemma)

In [ ]:
# If you want to use it outside of development you can clear the code by commenting the prints, How ever 
#I recomand adding logging to keep track of models responses and behavior
class CulturoGemma:
    def __init__(self, model_name:str="/kaggle/input/gemma2/transformers/gemma2_2b_mulitlingual_dpo/2", tavily_api:str="your-tavily-key", max_new_token:int=256):
        # You can use all variations of Gemma model family, You can add quantization, You can also modify the LLM loading
        # and response generation for using other frameworks like keras, Vllm,...
        # I also tested with functions like getting stock data and calculating simple math terms, Its works well
        # You can modify the function calling to use langchain or llamaindex or...
        try:
            self.model_name = model_name
            self.tavily_api = os.environ.get("TAVILY_KEY") or tavily_api

            if not self.model_name or not self.tavily_api:
                raise ValueError("Missing Model name Or API credentials.")

            self.model, self.tokenizer, error_message = self.load_model_and_tokenizer(model_name=self.model_name)
            if error_message:
              print(f"model loading error:{error_message}")
              raise Exception(error_message)
            self.max_new_token = max_new_token
            self.conversations = []

            # Define available functions
            self.available_functions = {
                "get_translation": self.get_translation,
                "get_web_result": self.get_web_result,
                "search_wikipedia": self.search_wikipedia,
                "translate": self.translate,
                "get_translation": self.get_translation,
                "fetch_url": self.get_url
            }

        except Exception as e:
            print(f"Chat Service initialization error: {e}")
            raise

    def set_system_instruction(self, instruction: str):
        """Allow updating the system instruction"""
        self.system_instruction = instruction

    def clean_response(self, response: str) -> str:
        """Clean the model's response by removing any system prompt repetition"""
        # Remove any system instruction repetition
        if "Your task is to assist users" in response:
            response = response.split("Current conversation:")[-1].strip()

        # Remove any role prefixes that might have been repeated
        response = re.sub(r'^(Assistant|User):', '', response).strip()

        return response
    def load_model_and_tokenizer(self, model_name:str):
        try:
            # Check if a GPU is available
            if not torch.cuda.is_available():
                return None, None, "GPU is not available. Please switch to GPU."

            tokenizer = AutoTokenizer.from_pretrained(model_name)

            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                device_map="auto"
            )
            print("Model And Tokenizer Loaded Successfully...")
            return model, tokenizer, None
        except Exception as e:
            print(f"Error loading model: {e}")
            return None, None, "Failed to load the model."

    def get_model_response(self, prompt:str) -> str:
      try:
        input_ids = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
          outputs = self.model.generate(**input_ids, max_new_tokens=self.max_new_token)
          response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response
      except Exception as e:
        print(f"Error generating response: {e}")
        return "Error generating response with the model."

    def get_translation(self, prompt:str) -> dict:
        """Get the translation of users prompt with google translate api"""
        try:
            translation = translate_to_english(prompt)
            return {"translation":translation}
        except Exception as e:
            return {"error": f"could not get the english translation: {str(e)}"}

    def translate(self, prompt:str) -> dict:
        """Get the translation from english prompt to detected target language"""
        try:
            translation = translate_to_target(prompt)
            return {"translation":translation}
        except Exception as e:
            return {"error": f"could not get the target translation: {str(e)}"}

    def get_web_result(self, query:str) -> dict:
        """Searches The web for a given query and returns an answer."""
        try:
            answer = get_answer_from_tavily(query)
            print(f"got answer from web for:{query}")
            return {"web_result": answer}
        except Exception as e:
            return {"error": f"could not get web answer: {str(e)}"}

    def get_url(self, urls):
        """fetches URLs."""
        try:
            url, content = fetch_url(urls)
            print(f"fetched for url:{urls}")
            return {"web_url":url, "extracted_content":content}
        except Exception as e:
            return {"error": f"could not fetch url: {str(e)}"}

    def search_wikipedia(self, query:str) -> dict:
        """Searches Wikipedia for a given query and returns a summary."""
        try:
            # Attempt to get the summary of the page directly
            return {"wikipedia_summary": wikipedia.summary(query, sentences=3)}
        except wikipedia.exceptions.PageError:
            return {"error": f"Wikipedia page not found for query: {query}"}
        except wikipedia.exceptions.DisambiguationError as e:
            return {"error": f"Multiple Wikipedia pages found for query: {query}. Be more specific.", "possible_titles": e.options}
        except Exception as e:
            return {"error": f"Error searching Wikipedia: {e}"}

    def execute_function(self, function_text: str) -> str:
        """Execute a function based on the text command"""
        try:
            # Extract function name and parameters using regex
            match = re.match(r'FUNCTION_CALL:\s*(\w+)\("([^"]*)"\)', function_text.strip())
            if not match:
                print(f"Failed to parse function call: {function_text}")
                return "Error: Invalid function call format. Please try again."

            function_name, param = match.groups()
            print(f"Executing function: {function_name} with param: {param}")

            if function_name not in self.available_functions:
                return f"Error: Unknown function {function_name}"

            result = self.available_functions[function_name](param)
            print(f"Function result: {result}")

            if isinstance(result, dict):
                if "error" in result:
                    return f"Error: {result['error']}"
                return str(result.get("web_result", result.get("extracted_content", result.get("translation", str(result)))))

            return str(result)

        except Exception as e:
            print(f"Function execution error: {e}")
            return f"Error executing function: {str(e)}"


    def format_conversation_history(self, messages):
        """Format conversation history with clear separation"""
        # Fixed instructions at the start of every prompt since Gemma dose not accept system role!
        formatted_prompt = """Your task is to assist users by searching for information and providing accurate responses.

For questions requiring current or factual information, you must first search using this format:
FUNCTION_CALL: get_web_result("search query")

EXAMPLES:
User: What is Thanksgiving?
Assistant: FUNCTION_CALL: get_web_result("What is Thanksgiving holiday history traditions")

User: What's happening in Iran?
Assistant: FUNCTION_CALL: get_web_result("current news Iran")

Other functions that you have access and you can use if needed:
For searching Wikipedia: FUNCTION_CALL: search_wikipedia("topic to search on wikipedia")
EXAMPLE:
user: What is Chaharshanbe Suri
Assistant: FUNCTION_CALL: search_wikipedia("Chaharshanbe Suri")

For fetching content from a specific URL(from user if provided): FUNCTION_CALL: fetch_url("URL")
For translating user input into English (for your better understanding): FUNCTION_CALL: get_translation("user's input in their language")
For translating your response into the user's language: FUNCTION_CALL: translate("your response in English")

Current conversation:
"""
        # Add conversation history
        for msg in messages:
            role = msg["role"]
            content = msg["content"]

            if role == "system":
                continue

            if role == "function":
                formatted_prompt += f"[Search Result: {content}]\n"
                continue

            role_prefix = "User" if role == "user" else "Assistant"
            formatted_prompt += f"{role_prefix}: {content}\n"

        # Add final prompt
        formatted_prompt += "Assistant: "
        return formatted_prompt

    def get_response(self, message):
        """Get a response from the assistant"""
        try:
            # Add user message
            self.conversations.append({"role": "user", "content": message})

            # Format conversation to avoid problems.
            formatted_prompt = self.format_conversation_history(self.conversations)
            print(f"\nFormatted prompt:\n{formatted_prompt}\n")

            # Get initial response(pre-function call)
            assistant_response = self.get_model_response(formatted_prompt)
            assistant_response = self.clean_response(assistant_response)
            print(f"\nInitial response:\n{assistant_response}\n")

            # Handle function calls
            if "FUNCTION_CALL:" in assistant_response:
                # Extract function call - take the first line that contains FUNCTION_CALL
                function_call = next(line for line in assistant_response.split('\n')
                                  if "FUNCTION_CALL:" in line).strip()
                print(f"\nDetected function call:\n{function_call}\n")

                # Execute function
                function_result = self.execute_function(function_call)
                print(f"\nFunction result:\n{function_result}\n")

                # Add to conversation history
                self.conversations.extend([
                    {"role": "assistant", "content": function_call},
                    {"role": "function", "content": function_result}
                ])

                # Get final response
                formatted_prompt = self.format_conversation_history(self.conversations)
                assistant_response = self.get_model_response(formatted_prompt)
                assistant_response = self.clean_response(assistant_response)
                print(f"\nFinal response:\n{assistant_response}\n")

            # Store final response
            self.conversations.append({"role": "assistant", "content": assistant_response})
            return assistant_response

        except Exception as e:
            print(f"Error in get_response: {e}")
            return f"I apologize, but I encountered an error: {str(e)}"# simple error handdling

**Explanation:**

* **`__init__`**: Initializes the model, tokenizer, conversation history, and sets the system prompt.
* **`get_model_response`**: Sends the formatted prompt to the Gemma model and returns the raw text response.
* **`execute_function`**: Parses the function call from the model's response and executes the corresponding function. It uses a `function_mapping` dictionary to link function names to their implementations.
* **`format_conversation_history`**: Formats the conversation history into a single string that can be fed to the model.
* **`get_response`**: This is the core method for handling user input:
    1. Appends the user's message to the conversation history.
    2. Translates the user's message to English for internal processing(if needed and choosed by the model).
    3. Detects the user's language.
    4. Formats the conversation history.
    5. Gets the initial response from the Gemma model.
    6. If the response contains a function call, it executes the function and gets an updated response.
    7. Translates the final response back to the user's language(if needed and choosed by the model).
    8. Appends the assistant's response to the conversation history.


## Initialzing the `CulturoGemma`
By running the cell below we initialize the `CulturoGemma`.

In [ ]:
try:
  culturo_gemma = CulturoGemma()
  print("Culturo Gemma is ready to go...")
except Exception as e:
  print(f"Error initializing Culturo Gemma: {e}")

In [ ]:
response = culturo_gemma.get_response("What is Greenery Day?")
print(f"response: \n {response}")

## Creating a Simple Command-Line Interface

To test our agent, we can create a simple command-line interface using Jupyter Notebook.

In [ ]:
print("Welcome to the Culturo Gemma! Type 'EnD' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "end":
        print("Chat ended.")
        break

    try:
        response = culturo_gemma.get_response(user_input)
        print(f"AI: {response}")
        print("\n ------------------------ \n")
    except Exception as e:
        print(f"Error processing your request: {e}")

**Explanation:**

* We create an instance of the `CulturoGemma`.
* A `while` loop allows for continuous interaction until the user types "EnD".
* The `input()` function gets the user's message.
* The `culturo_gemma.get_response()` method processes the input and generates the AI's response.
* The AI's response is printed to the console.
* Error handling is included to catch any issues during the process.

## Building a Streamlit Interface

For a more user-friendly experience, we can create a web interface using Streamlit.

In [ ]:
%%writefile streamlit_app.py
import os
import io
import re
import json
import torch
import wikipedia
import streamlit as st
from tavily import TavilyClient
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set the environment variables for Kaggle and tavily search.
# from kaggle_secrets import UserSecretsClient if you use kaggle
# from google.colab import userdata if you use google colab
#import getpass if you use jupyter notebook
os.environ["KAGGLE_USERNAME"] = "your-username"# or UserSecretsClient().get_secret(KAGGLE_USERNAME) or userdata.get(KAGGLE_USERNAME) or getpass.getpass("Enter your KAGGLE_USERNAME: ")
os.environ["KAGGLE_KEY"] = "kaggle-api-key" # or UserSecretsClient().get_secret(KAGGLE_KEY) or userdata.get(KAGGLE_KEY) or getpass.getpass("Enter your  KAGGLE_KEY: ")
os.environ["TAVILY_KEY"] = "tavily-api-key" # or UserSecretsClient().get_secret(TAVILY_KEY) or userdata.get(TAVILY_KEY) or getpass.getpass("Enter your TAVILY_KEY: ")

# Initialize API clients using environment variables
try:
    tavily_client = TavilyClient(api_key=os.environ["TAVILY_KEY"])
except KeyError:
    print("Error: TAVILY_KEY environment variable not set.")
    tavily_client = None

def get_answer_from_tavily(query):
    """ This function gets query as str and passes the extracted answer as str"""
    try:
        answer = tavily_client.qna_search(query=query)
        # Assuming the response has an 'answer' key
        return answer
    except Exception as e:
        print(f"Error fetching answer from Tavily: {e}")
        return f"Error fetching answer: {str(e)}"

def fetch_url(urls):
    """ this function fetches the user url and returns the extracted content"""
    try:
        extract_response = tavily_client.extract(urls=urls)
        for result in extract_response.get("results", []):
            return result["url"], result["raw_content"]
        return "No results found", "No content found"
    except Exception as e:
        print(f"Error fetching URL content: {e}")
        return f"Error fetching URL: {str(e)}", f"Error fetching content: {str(e)}"
use_google_translate = False
if use_google_translate:
    from google.cloud import translate_v2 as translate
    from google.api_core.exceptions import GoogleAPIError
    # Create a client with your credentials (replace with your service account key file)
    translate_client = translate.Client.from_service_account_json('path/to/your/service_account_key.json')

detected_language = None  # Initialize a global variable to store the detected language

def detect_language(prompt):
    """Detects the language of the given prompt, handling potential errors."""
    global detected_language
    try:
        # Check if translation functionality is enabled
        if use_google_translate:
            result = translate_client.detect_language(prompt)
            detected_language = result['language']
            return detected_language
        else:
            # Return None or a default language if translation is disabled
            return None
    except GoogleAPIError as e:
        print(f"Error detecting language: {e}")
        return None

def translate_to_english(prompt):
  """Translates the given prompt to English, handling potential errors."""
  try:
    if use_google_translate:
      result = translate_client.translate(prompt, target_language='en')
      return result['translatedText']
    else:
      return None  # Return None if translation is disabled
  except GoogleAPIError as e:
    print(f"Error translating to English: {e}")
    return prompt  # Return the original prompt on failure

def translate_to_target(prompt):
  """Translates the given prompt to the previously detected target language, handling errors."""
  global detected_language
  try:
    if use_google_translate and detected_language:
      result = translate_client.translate(prompt, target_language=detected_language)
      return result['translatedText']
    else:
      return "Target language not detected Or google cloud api not provided. Please call detect_language() first and `use_google_translate=True`."
  except GoogleAPIError as e:
    print(f"Error translating to target language: {e}")
    return prompt  # Return the original prompt on failure
      
# If you want to use it outside of development you can clear the code by commenting the prints, How ever 
#I recomand adding logging to keep track of models responses and behavior
class CulturoGemma:
    def __init__(self, model_name:str="/kaggle/input/gemma2/transformers/gemma2_2b_mulitlingual_dpo/2", tavily_api:str="your-tavily-key", max_new_token:int=256):
        # You can use all variations of Gemma model family, You can add quantization, You can also modify the LLM loading
        # and response generation for using other frameworks like keras, Vllm,...
        # I also tested with functions like getting stock data and calculating simple math terms, Its works well
        # You can modify the function calling to use langchain or llamaindex or...
        try:
            self.model_name = model_name
            self.tavily_api = os.environ.get("TAVILY_KEY") or tavily_api

            if not self.model_name or not self.tavily_api:
                raise ValueError("Missing Model name Or API credentials.")

            self.model, self.tokenizer, error_message = self.load_model_and_tokenizer(model_name=self.model_name)
            if error_message:
              print(f"model loading error:{error_message}")
              raise Exception(error_message)
            self.max_new_token = max_new_token
            self.conversations = []

            # Define available functions
            self.available_functions = {
                "get_translation": self.get_translation,
                "get_web_result": self.get_web_result,
                "search_wikipedia": self.search_wikipedia,
                "translate": self.translate,
                "get_translation": self.get_translation,
                "fetch_url": self.get_url
            }

        except Exception as e:
            print(f"Chat Service initialization error: {e}")
            raise

    def set_system_instruction(self, instruction: str):
        """Allow updating the system instruction"""
        self.system_instruction = instruction

    def clean_response(self, response: str) -> str:
        """Clean the model's response by removing any system prompt repetition"""
        # Remove any system instruction repetition
        if "Your task is to assist users" in response:
            response = response.split("Current conversation:")[-1].strip()

        # Remove any role prefixes that might have been repeated
        response = re.sub(r'^(Assistant|User):', '', response).strip()

        return response
    def load_model_and_tokenizer(self, model_name:str):
        try:
            # Check if a GPU is available
            if not torch.cuda.is_available():
                return None, None, "GPU is not available. Please switch to GPU."

            tokenizer = AutoTokenizer.from_pretrained(model_name)

            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                device_map="auto"
            )
            print("Model And Tokenizer Loaded Successfully...")
            return model, tokenizer, None
        except Exception as e:
            print(f"Error loading model: {e}")
            return None, None, "Failed to load the model."

    def get_model_response(self, prompt:str) -> str:
      try:
        input_ids = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
          outputs = self.model.generate(**input_ids, max_new_tokens=self.max_new_token)
          response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response
      except Exception as e:
        print(f"Error generating response: {e}")
        return "Error generating response with the model."

    def get_translation(self, prompt:str) -> dict:
        """Get the translation of users prompt with google translate api"""
        try:
            translation = translate_to_english(prompt)
            return {"translation":translation}
        except Exception as e:
            return {"error": f"could not get the english translation: {str(e)}"}

    def translate(self, prompt:str) -> dict:
        """Get the translation from english prompt to detected target language"""
        try:
            translation = translate_to_target(prompt)
            return {"translation":translation}
        except Exception as e:
            return {"error": f"could not get the target translation: {str(e)}"}

    def get_web_result(self, query:str) -> dict:
        """Searches The web for a given query and returns an answer."""
        try:
            answer = get_answer_from_tavily(query)
            print(f"got answer from web for:{query}")
            return {"web_result": answer}
        except Exception as e:
            return {"error": f"could not get web answer: {str(e)}"}

    def get_url(self, urls):
        """fetches URLs."""
        try:
            url, content = fetch_url(urls)
            print(f"fetched for url:{urls}")
            return {"web_url":url, "extracted_content":content}
        except Exception as e:
            return {"error": f"could not fetch url: {str(e)}"}

    def search_wikipedia(self, query:str) -> dict:
        """Searches Wikipedia for a given query and returns a summary."""
        try:
            # Attempt to get the summary of the page directly
            return {"wikipedia_summary": wikipedia.summary(query, sentences=3)}
        except wikipedia.exceptions.PageError:
            return {"error": f"Wikipedia page not found for query: {query}"}
        except wikipedia.exceptions.DisambiguationError as e:
            return {"error": f"Multiple Wikipedia pages found for query: {query}. Be more specific.", "possible_titles": e.options}
        except Exception as e:
            return {"error": f"Error searching Wikipedia: {e}"}

    def execute_function(self, function_text: str) -> str:
        """Execute a function based on the text command"""
        try:
            # Extract function name and parameters using regex
            match = re.match(r'FUNCTION_CALL:\s*(\w+)\("([^"]*)"\)', function_text.strip())
            if not match:
                print(f"Failed to parse function call: {function_text}")
                return "Error: Invalid function call format. Please try again."

            function_name, param = match.groups()
            print(f"Executing function: {function_name} with param: {param}")

            if function_name not in self.available_functions:
                return f"Error: Unknown function {function_name}"

            result = self.available_functions[function_name](param)
            print(f"Function result: {result}")

            if isinstance(result, dict):
                if "error" in result:
                    return f"Error: {result['error']}"
                return str(result.get("web_result", result.get("extracted_content", result.get("translation", str(result)))))

            return str(result)

        except Exception as e:
            print(f"Function execution error: {e}")
            return f"Error executing function: {str(e)}"


    def format_conversation_history(self, messages):
        """Format conversation history with clear separation"""
        # Fixed instructions at the start of every prompt since Gemma dose not accept system role!
        formatted_prompt = """Your task is to assist users by searching for information and providing accurate responses.

For questions requiring current or factual information, you must first search using this format:
FUNCTION_CALL: get_web_result("search query")

EXAMPLES:
User: What is Thanksgiving?
Assistant: FUNCTION_CALL: get_web_result("What is Thanksgiving holiday history traditions")

User: What's happening in Iran?
Assistant: FUNCTION_CALL: get_web_result("current news Iran")

Other functions that you have access and you can use if needed:
For searching Wikipedia: FUNCTION_CALL: search_wikipedia("topic to search on wikipedia")
EXAMPLE:
user: What is Chaharshanbe Suri
Assistant: FUNCTION_CALL: search_wikipedia("Chaharshanbe Suri")

For fetching content from a specific URL(from user if provided): FUNCTION_CALL: fetch_url("URL")
For translating user input into English (for your better understanding): FUNCTION_CALL: get_translation("user's input in their language")
For translating your response into the user's language: FUNCTION_CALL: translate("your response in English")

Current conversation:
"""
        # Add conversation history
        for msg in messages:
            role = msg["role"]
            content = msg["content"]

            if role == "system":
                continue

            if role == "function":
                formatted_prompt += f"[Search Result: {content}]\n"
                continue

            role_prefix = "User" if role == "user" else "Assistant"
            formatted_prompt += f"{role_prefix}: {content}\n"

        # Add final prompt
        formatted_prompt += "Assistant: "
        return formatted_prompt

    def get_response(self, message):
        """Get a response from the assistant"""
        try:
            # Add user message
            self.conversations.append({"role": "user", "content": message})

            # Format conversation to avoid problems.
            formatted_prompt = self.format_conversation_history(self.conversations)
            print(f"\nFormatted prompt:\n{formatted_prompt}\n")

            # Get initial response(pre-function call)
            assistant_response = self.get_model_response(formatted_prompt)
            assistant_response = self.clean_response(assistant_response)
            print(f"\nInitial response:\n{assistant_response}\n")

            # Handle function calls
            if "FUNCTION_CALL:" in assistant_response:
                # Extract function call - take the first line that contains FUNCTION_CALL
                function_call = next(line for line in assistant_response.split('\n')
                                  if "FUNCTION_CALL:" in line).strip()
                print(f"\nDetected function call:\n{function_call}\n")

                # Execute function
                function_result = self.execute_function(function_call)
                print(f"\nFunction result:\n{function_result}\n")

                # Add to conversation history
                self.conversations.extend([
                    {"role": "assistant", "content": function_call},
                    {"role": "function", "content": function_result}
                ])

                # Get final response
                formatted_prompt = self.format_conversation_history(self.conversations)
                assistant_response = self.get_model_response(formatted_prompt)
                assistant_response = self.clean_response(assistant_response)
                print(f"\nFinal response:\n{assistant_response}\n")

            # Store final response
            self.conversations.append({"role": "assistant", "content": assistant_response})
            return assistant_response

        except Exception as e:
            print(f"Error in get_response: {e}")
            return f"I apologize, but I encountered an error: {str(e)}"# simple error handdling


# Initialize ChatService in session state
if 'culturo_gemma' not in st.session_state:
    st.session_state.culturo_gemma = CulturoGemma()

# Initialize chat history in session state
if 'messages' not in st.session_state:
    st.session_state.messages = []

st.title("Multilingual & Culturally Aware AI Agent")

user_input = st.text_input("You:", key="user_input")

if st.button("Send"):
    if user_input:
        st.session_state.messages.append({"role": "user", "content": user_input})
        try:
            ai_response = st.session_state.culturo_gemma.get_response(user_input)
            st.session_state.messages.append({"role": "assistant", "content": ai_response})
        except Exception as e:
            st.session_state.messages.append({"role": "error", "content": f"Error: {e}"})
        st.session_state["user_input"] = ""

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

**Explanation:**

* We import the necessary libraries, including `streamlit`.
* We initialize the `ChatService` and the chat `messages` list in Streamlit's `session_state`. This ensures that the state of the application is preserved across interactions.
* `st.title()` sets the title of the web application.
* `st.text_input()` creates a text input field for the user.
* `st.button()` creates a "Send" button.
* When the button is clicked, the user's input is processed by the `chat_service.get_response()` method, and the response is added to the chat history.
* `st.chat_message()` is used to display the chat messages with different styling based on the role (user or assistant).

To run the Streamlit application, save this code as a Python file (e.g., `streamlit_app.py`) and run the following command in your terminal:

```bash
streamlit run streamlit_app.py
```

## Naming the Agent: CulturoGemma

I've poured effort into building an AI agent that's not just intelligent but also adept at navigating the complexities of language and culture. To give our creation a fitting identity, I've chosen the name **CulturoGemma**.

This name is a deliberate blend of two key aspects of our agent:

* **Culturo:** This prefix directly emphasizes the agent's **cultural awareness** and sensitivity. It signifies the agent's ability to understand and respect diverse customs, traditions, and perspectives. The "Culturo" element highlights our focus on building an AI that can engage thoughtfully and respectfully with users from various backgrounds.

* **Gemma:** This suffix clearly identifies the underlying **power and intelligence** of the agent, stemming from the Gemma large language model. By including "Gemma," we acknowledge the technological foundation that enables the agent's sophisticated language processing and generation capabilities.

## Future path:
**What else can we do?**

So we can do anything but I suggest these(I would do them if I have the time):
* 1- a sort of ReACT agent, So it can call multiple functions in a single response
* 2- adding more functions
* 3- fine-tuning the model for multi-lingual instruct and tool calling
* 4- adding voice featuers(But this is out of our current scope since we deal with the model not the web or app, It can be achieved easily, Take a look at one of my projects, [repo](https://github.com/Mhdaw/All-chat)
* 5- incorprating RAG as well, You can see an example in one of my notebooks published.
* 6- using langchain and other libraries to make the progress more reliable

### My Gemma cookbook:
I made this repo and uploaded all of my projects:
https://github.com/Mhdaw/Gemma2

## Conclusion

This notebook has demonstrated the process of building a sophisticated AI chat agent capable of engaging in multilingual and culturally sensitive conversations. By leveraging powerful language models, external APIs, and careful design, we can create AI assistants that are more inclusive and effective in a global context. Further improvements could include:

* **More sophisticated error handling and user feedback.**
* **Integration with more diverse information sources.**
* **Fine-tuning the Gemma model for specific cultural contexts.**
* **Implementing user authentication and personalized experiences.**

This project provides a solid foundation for building advanced AI agents that can communicate and understand the world in a richer and more nuanced way.